In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

###  Event stamps ( array I )

### TOs and ITIs:

#### 7   = Time-out, no lever response choice in 10 sec
#### 19  = Time-out, no right lever response forced in 10 seconds
#### 20  = Time-out, no left lever response forced in 10 seconds

#### 16  = ITI over

### LPs:

#### 17  = Initial forced trial lever press right
#### 18  = Initial forced trial lever press left
#### 8   = Lever choice right
#### 9   = Lever choice left

#### 5   = Reinforcement delivery

### HEs:

#### 10  = Initial head entry in trial   **** IN VERSION 22_BERNSTEIN THIS IS SIMULATED AFTER 1 SECOND ** VERSION 23 REINSTATED
#### 11  = Subsequent head entries




-   1   = Rt lever response in FR (if FR greater than 1 otherwise see codes 8,9,17,18,21,22)
-   2   = Lt lever response (if FR greater than 1 otherwise see codes 8,9,17,18,21,22)
-   3   = Rewarded rt lever response
-   4   = Rewarded lt lever response
-   6   = Time-out, no initial head-entry in 10 sec   **** REMOVED FOR VERSION 22_BERNSTEIN ** REINSTATED VERSION 23
-   12  = Right lever press during time-out
-   13  = Left lever press during time-out
-   14  = Head entry during time-out
-   15  = Time-out over
-   21  = Forced trial right lever
-   22  = Forced trail left lever
-   23  = Choice trial
-   24  = Head entry during ITI
-   25  = ICS signal
-   26  = Extinction trial
-   27  = Time-out, no press in A(22) minutes
-   28  = Time-out, did not complete FR in A(21) minutes
-   100 = Session termination




## Variables

Use these to check that correct parameters were entered into the Med-PC program for each file

#### A(4) = Reward on right lever
#### A(5) = Reward on left lever
#### A(24) = Probabilistic lever (0 = None, 1 = Right, 2 = Left, 3 = Both)
#### A(10) = Forced or choice (1 = forced, 2 = choice)

- A(1) = Intertrial interval calculated 
- A(2) = FR for right lever
- A(3) = FR for left lever
- A(6) = Delay on right lever
- A(7) = Delay on left lever
- A(8) = Lever choice (1 = right, -1 = left lever)
- A(9) = Maximum session length
- A(11) = Time to complete FR in seconds 
- A(12) = Time-out length
- A(13) = Maximum FR
- A(14) = Maximum intertrial interval (60 seconds default)
- A(15) = Minimum intertrial interval (30 seconds default)
- A(16) = Counter for trial number
- A(17) = Maximum number of trials (48 default)
- A(18) = Time limit defining a miss (10 seconds default)
- A(19) = Counter for completed right lever FRs
- A(20) = Counter for completed left lever FRs
- A(21) = Time-limit for completing an FR (minutes)
- A(22) = Time-limit for between presses; i.e., if a press is not given at 
           least every A(22) minutes, a miss occurs
- A(23) = Repeat missed trials? (1:YES   2:NO)
- A(25) = Chosen from P list of reinforcement or no reinforcement
- A(26) = Counter for reinf probs list

In [2]:
def get_file_info(file_path_name):
    # takes in a file path, finds the date, animal_number, session, session number, and sub_session, saves each accordingly
    import re
    
    path_split = file_path_name.split('/')
    file_split = path_split[-1].split('_')
    
    for row in file_split:
        if re.search(r'^\d\d\d\d\d\d$', row):
            date = row
        if re.search(r'^\d\d\d$', row):
            animal_number = row
        if re.search(r'((F|f)orce)|((C|c)hoice)', row):
            sub_session = row
        if re.search(r'^\d$', row):
            session_num = row
    
    try:
        sub_session
    except:
        sub_session = np.nan
        
    try:
        session = str(file_split[0] + '_' + session_num)
    except: 
        session = file_split[0]

    return date, animal_number, session, sub_session

In [3]:
def make_array_from_path (file_path_name):
    # takes in file path, creates array (each row of array is a row from original med file)
    import numpy as np
    import pandas as pd
    
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [4]:
def get_events_and_times(array, event_letter, time_letter):
    #takes in array and the letters Med PC program uses to signify an event_stamp and time_stamp; 
    #finds all event_stamps and corresponding time_stamps; creats pandas dataframe
    import re
    import numpy as np
    import pandas as pd

    i = 0
    event_data = []
    time_data = []
    while i < len(array):
        if re.search(str(event_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    event_data.append(float(element))
                i = i + 1
        if re.search(str(time_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    time_data.append(float(element))
                i = i + 1
        i = i + 1
    
    eandt = np.column_stack((event_data, time_data))
    es_et = pd.DataFrame(data = eandt)
    es_et.columns = ['event_stamp', 'time_stamp']
    
    return es_et

In [5]:
def get_trial_times_two(events_and_times, start_marker, ITI_marker, TO_rt_marker, TO_lt_marker, TO_choice_marker):
    #takes in dataframe of events and corresponding time_stamps and the Med PC program code for trial related markers
    #e.g. conditioned stimulus onset and offset, ITI onset and offset
    #finds time_stamps for each start and end, puts into new dataframe
    import numpy as np
    import pandas as pd

    trial_starts = events_and_times[events_and_times['event_stamp'] == start_marker]['time_stamp'].values
    trial_ends = events_and_times[events_and_times['event_stamp'] == ITI_marker]['time_stamp'].values
    trial_lp_TOs = events_and_times[(events_and_times['event_stamp'] == TO_rt_marker) | (events_and_times['event_stamp'] == TO_lt_marker) | (events_and_times['event_stamp'] == TO_choice_marker)]['time_stamp'].values
    
    trial_ends_updated = np.sort(np.concatenate((trial_ends, trial_lp_TOs), axis=0))
    
    times = pd.DataFrame(data = [trial_starts, trial_ends_updated])
    times = times.T
    times.columns = ['trial_start', 'trial_end']
    
    return times

In [6]:
def create_trials_df(events_and_times, trial_times):
    #takes in df of events and times and df of trial times
    #creates tidy df containing events and times according to trials
    i = 0
    trial_data = pd.DataFrame()
    
    while i < len(trial_times):
        data = events_and_times[(events_and_times['time_stamp'] >= trial_times['trial_start'][i]) & (events_and_times['time_stamp'] <= trial_times['trial_end'][i])]
        data['trial_#'] = [i]*len(data)
        data['trial_start'] = trial_times['trial_start'][i]
        data['trial_end'] = trial_times['trial_end'][i]
        trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
        i = i + 1
    
    return trial_data

In [7]:
def count(trial_data, event_num):
    #get count of event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    count = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        count.append(len(data[data['event_stamp'] == event_num]))
        i = i + 1
    
    return count

In [8]:
def latency(trial_data, event_num):
    #get latency of time between trial_start and event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    latency_1st = []
    latency_ave = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        data = data[data['event_stamp'] == event_num]
        if len(data) < 1:
            latency_1st.append(0.0)
            latency_ave.append(0.0)
        else:
            latency_1st.append((data['time_stamp'] - data['trial_start']).values[0]/100)
            latency_ave.append((data['time_stamp'] - data['trial_start']).values.mean()/100)
        i = i + 1
    
    return latency_1st, latency_ave

In [9]:
def create_session_df(trial_data, event_dic):
    
    trials = len((trial_data['trial_#'].unique()))
    
    #get HE_wait times
    times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['trial_start'].unique(), trial_data.groupby('trial_#')['trial_end'].unique()])
    times_df = times_df.T
    
    i = 0
    HE_waits = []
    while i < times_df.shape[0]:
        if (i == 0) | (i == times_df.shape[0] - 1):
            HE_waits.append(np.nan)
            i += 1
        else:
            HE_wait = times_df.iloc[i]['trial_start'] - times_df.iloc[i-1]['trial_end']
            HE_waits.append(int(HE_wait/100))
            i += 1
    
    session_data = pd.DataFrame(index=np.arange(0,trials))

    for name, event in event_dic.items():
        session_data[str(name + '_count')] = count(trial_data, event)
        session_data[str(name + '_latency_1st')], session_data[str(name + '_latency_ave_bit')] = latency(trial_data, event)
    
    session_data['trial_num'] = np.arange(session_data.shape[0])
    session_data['HE_wait'] = HE_waits
    
    return session_data

In [10]:
def get_variables(array, variable_letter):
    #takes in array and the letter Med PC program uses to encode variables (parameters for behavioral program entered at time of exp)
    import re
    import numpy as np
    import pandas as pd

    i = 0
    variable_data = []
    while i < len(array):
        if re.search(str(variable_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    variable_data.append(float(element))
                i = i + 1
        i = i + 1
    
    variable_df = pd.DataFrame(data = variable_data)
    variable_df.columns = ['variables']
    
    return variable_df

In [11]:
def check_parameter_entry(session_data, animal_info):
    
    #list of session names that we want to check (eg any session that used a probability)
    probabilities = ['75', '50', '25']
    
    animal = int(session_data['animal_number'].unique())
    
    #for 100 probability
    if session_data['session'].unique() == '100':
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 1:
            session_data['rt_reward'] = 2
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 0
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 2
            session_data['prob_lever'] = 0
            
    #for probabilities 75, 50, 25
    elif session_data['session'].unique() in probabilities:
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 1:
            session_data['rt_reward'] = 2
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 1
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 2
            session_data['prob_lever'] = 2
            
    #for 0 probability trial
    elif session_data['session'].unique() == '0':
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 1:
            session_data['rt_reward'] = 0
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 0
        if animal_info[animal_info['animal'] == animal]['high_reward'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 0
            session_data['prob_lever'] = 0
            
    else:
        return print('This is not a probability session.') 
            
    #assume entries were false until check determines otherwise
    rt_reward = False
    lt_reward = False
    force_choice = False
    prob_lever = False
    
    #get variables data containing parameter entries
    variables_df = get_variables(array, "A")

    #use varibales data and session data to determine if the correct parameters were entered
    if float(variables_df.iloc[4].values) in session_data['rt_reward'].unique():
        rt_reward = True

    if float(variables_df.iloc[5].values) in session_data['lt_reward'].unique():
        lt_reward = True
    
    if float(variables_df.iloc[24].values) in session_data['prob_lever'].unique():
        prob_lever = True
        
    if session_data['sub_session'].unique() == 'force':
        sub_session = 1
    if session_data['sub_session'].unique() == 'choice':
        sub_session = 2 
    if float(variables_df.iloc[10].values) == sub_session:
        force_choice = True
    
    #print results
    return print('rt_reward:', rt_reward, '\n', 'lt_reward:', lt_reward, '\n', 'force_choice:', force_choice, '\n', 'prob_lever:', prob_lever, '\n')
    

In [12]:
path_1 = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape'

session_list_1 = os.listdir(path_1)
print(len(session_list_1))
session_list_1[0:5]

198


['100_choice_682_190620',
 '100_choice_682_190621',
 '100_choice_683_190619',
 '100_choice_683_190620',
 '100_choice_683_190621']

In [13]:
file_paths = []
for file in session_list_1:
    file_path = path_1 + '/' + file
    file_paths.append(file_path)

print(len(file_paths))
file_paths[0:5]

198


['C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_682_190620',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_682_190621',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190619',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190620',
 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190621']

In [14]:
#read animal info (lever assignment, group etc) into df

path_animal_info = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/Med boxes/PD_round1.xlsx'
    
animal_info = pd.read_excel(path_animal_info)
animal_info = pd.DataFrame(data = animal_info)

animal_info.head()

,animal,high_reward
0,682,1
1,683,1
2,684,2
3,685,2
4,686,2


In [30]:
%%time
final_data = pd.DataFrame()

#these are the event_stamps we want analyzed
event_dic = {'reinforcement': 5, 'right_lever_choice': 8, 'left_lever_choice': 9, 'right_lever_force': 17, \
             'left_lever_force': 18, 'HE_trial': 11}

run_variables = True

for file in file_paths:
    print(file)
    
    #proccess file
    date, animal_number, session, sub_session = get_file_info(file)
    array = make_array_from_path(file)
    events_and_times = get_events_and_times(array, "I", "T")
    trial_times = get_trial_times_two(events_and_times, 10, 16, 19, 20, 7)
    trial_data = create_trials_df(events_and_times, trial_times)
    session_data = create_session_df(trial_data, event_dic)
    
    #create pandas tidy df
    session_data['date'] = [date]*(session_data.shape[0])
    session_data['animal_number'] = [animal_number]*(session_data.shape[0])
    session_data['session'] = [session]*(session_data.shape[0])
    session_data['sub_session'] = [sub_session]*(session_data.shape[0])
    
    try:
        high_reward_lever = animal_info[animal_info['animal'] == int(animal_number)]['high_reward'].values[0]
        session_data['high_reward_lever'] = [high_reward_lever]*(session_data.shape[0])
    except:
        session_data['high_reward_lever'] = np.nan
    
    #run varibale check
    if run_variables == True:
        check_parameter_entry(session_data, animal_info)
    
    if run_variables == False:
        session_data['rt_reward'] = np.nan
        session_data['lt_reward'] = np.nan
        session_data['prob_lever'] = np.nan
    
    #combine lever force and choice data (use sub_session column to groupby instead)
    if sub_session == 'force':
        session_data.rename({'right_lever_force_count': 'right_lever_count',
                          'left_lever_force_count': 'left_lever_count',
                          'right_lever_force_latency_1st': 'right_lever_latency',
                          'left_lever_force_latency_1st': 'left_lever_latency',}, axis='columns', inplace=True)
    elif sub_session == 'choice':  
        session_data.rename({'right_lever_choice_count': 'right_lever_count',
                          'left_lever_choice_count': 'left_lever_count',
                          'right_lever_choice_latency_1st': 'right_lever_latency',
                          'left_lever_choice_latency_1st': 'left_lever_latency',}, axis='columns', inplace=True)
    else:
        pass

    final_data = pd.concat([final_data, session_data], axis = 0, ignore_index=True)

final_data = final_data[['date', 'animal_number', 'session', 'sub_session', 'trial_num', 'high_reward_lever', 
                         'reinforcement_count', 'right_lever_count', 'right_lever_latency', 
                         'left_lever_count', 'left_lever_latency', 
                         'HE_trial_count', 'HE_trial_latency_1st', 'HE_wait', 'rt_reward', 'lt_reward', 'prob_lever']]

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_682_190620
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_682_190621
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190619
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190620
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_choice_683_190621
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Docume

rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_force_688_190621
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_force_689_190621
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_force_690_190621
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_force_691_190619
rt_reward: True 
 lt_reward: True 
 force_choice: True 
 prob_lever: True 

C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/100_force_692_190620
rt_reward: True 
 lt_reward: T

This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_689_190618
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_690_190618
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_691_190614
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_692_190617
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_693_190617
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_2_choice_694_190614
This is not a probability session.
C:/Users/Schindle

This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_684_190617
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_685_190618
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_686_190617
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_687_190618
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_688_190618
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_3_force_689_190619
This is not a probability session.
C:/Users/Schindler/Docu

This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_4_force_699_190619
This is not a probability session.
C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_4_force_700_190620
This is not a probability session.
Wall time: 8min 50s


In [31]:
final_data.to_csv('shape.csv')

In [33]:
final_data.head(50)

,date,animal_number,session,sub_session,trial_num,high_reward_lever,reinforcement_count,right_lever_count,right_lever_latency,left_lever_count,left_lever_latency,HE_trial_count,HE_trial_latency_1st,HE_wait,rt_reward,lt_reward,prob_lever
0,190620,682,100,choice,0,1,1,0.0,0.00,1.0,1.15,2,0.05,NaN,2.0,1.0,0.0
1,190620,682,100,choice,1,1,1,0.0,0.00,1.0,1.00,1,0.10,10.0,2.0,1.0,0.0
2,190620,682,100,choice,2,1,1,0.0,0.00,1.0,1.48,1,0.06,3.0,2.0,1.0,0.0
3,190620,682,100,choice,3,1,1,0.0,0.00,1.0,0.85,1,0.14,10.0,2.0,1.0,0.0
4,190620,682,100,choice,4,1,1,0.0,0.00,1.0,0.88,1,0.04,0.0,2.0,1.0,0.0
5,190620,682,100,choice,5,1,1,0.0,0.00,1.0,1.10,1,0.13,4.0,2.0,1.0,0.0
6,190620,682,100,choice,6,1,1,0.0,0.00,1.0,1.05,0,0.00,7.0,2.0,1.0,0.0
7,190620,682,100,choice,7,1,1,0.0,0.00,1.0,0.76,1,0.04,2.0,2.0,1.0,0.0
8,190620,682,100,choice,8,1,1,0.0,0.00,1.0,1.07,0,0.00,8.0,2.0,1.0,0.0
9,190620,682,100,choice,9,1,1,0.0,0.00,1.0,1.48,0,0.00,10.0,2.0,1.0,0.0


In [ ]:
#single file option for trouble shooting

run_variables = False

file_path = 'C:/Users/Schindler/Documents/Schindler_Lab/Data/Behavior/Med boxes/probability_discounting/shape/Shape_4_choice_683_190618'
print('file_path:', '\n', file_path, '\n')

date, animal_number, session, sub_session = get_file_info(file_path)
print('date:', date, 'animal_number:', animal_number, 'session:', session, 'sub_session:', sub_session, '\n')

array = make_array_from_path(file_path)
print(array)

events_and_times = get_events_and_times(array, "I", "T")
print(events_and_times.head(), '\n')

trial_times = get_trial_times_two(events_and_times, 10, 16, 19, 20, 7)
print(trial_times.head(), '\n')

trial_data = create_trials_df(events_and_times, trial_times)
print(trial_data.head(), '\n')

bit_dic = {'reinforcement': 5, 'right_lever_choice': 8, 'left_lever_choice': 9, 'right_lever_force': 17, 'left_lever_force': 18, 'HE_trial': 11}
session_data = create_session_df(trial_data, bit_dic)

session_data['date'] = [date]*(session_data.shape[0])
session_data['animal_number'] = [animal_number]*(session_data.shape[0])
session_data['session'] = [session]*(session_data.shape[0])
session_data['sub_session'] = [sub_session]*(session_data.shape[0])

if run_variables == True:
        check_parameter_entry(session_data, animal_info)
    
if run_variables == False:
    session_data['rt_reward'] = np.nan
    session_data['lt_reward'] = np.nan
    session_data['prob_lever'] = np.nan
    
try:
    session_data['high_reward_lever'] = animal_info[animal_info['animal'] == animal_number]['high_reward']
except:
    session_data['high_reward_lever'] = np.nan

final_data = pd.DataFrame()

final_data = pd.concat([final_data, session_data], axis = 0, ignore_index=True)

final_data = final_data[['date', 'animal_number', 'session', 'sub_session', 'trial_num', 'high_reward_lever', 
                         'reinforcement_count', 'right_lever_force_count', 'right_lever_force_latency_1st', 
                         'left_lever_force_count', 'left_lever_force_latency_1st', 'right_lever_choice_count', 
                         'right_lever_choice_latency_1st', 'left_lever_choice_count', 'left_lever_choice_latency_1st', 
                         'HE_trial_count', 'HE_trial_latency_1st', 'HE_wait', 'rt_reward', 'lt_reward', 'prob_lever']]